# 01. 분석에 필요한 데이터 Merge

In [1]:
import pandas as pd
import time

In [2]:
# 데이터 전처리에 걸리는 시간 체크
tic = time.time()

# na_values: '~' 데이터 nan 처리
data1 = pd.read_csv('./data/smartcard20171019_before_Chained.csv', encoding = 'cp949', na_values = '~')
data2 = pd.read_csv('./data/smartcard20171019.csv', encoding = 'cp949', na_values = '~')
print('Loading', time.time() - tic)
print('data1 shape: ', data1.shape)
print('data1 shape: ', data2.shape)

Loading 51.513899087905884
data1 shape:  (10338082, 19)
data1 shape:  (10639365, 17)


In [108]:
data2 = data2[['카드번호','노선ID','승차일시','하차일시']]
data2['on_hour'] = data2['승차일시'] // 1e+4 % 100
data2['on_minute'] = data2['승차일시'] // 100 % 100
data2['on_second'] = data2['승차일시'] % 100

In [109]:
# 데이터 전처리에 걸리는 시간 체크
tic = time.time()

df = pd.merge(data1, data2, left_on = ['card_name','on_hour','on_minute','on_second'],
             right_on = ['카드번호','on_hour','on_minute','on_second'])
print('Join', time.time() - tic)

Join 17.56479787826538


In [110]:
del data1, data2

In [111]:
df = df.sort_values(['card_name','on_hour','on_minute','on_second'], ignore_index = True)
df.head()

,card_name,route_id,on_area_name,on_lon,on_lat,on_dong,on_hour,on_minute,on_second,off_area_name,off_lon,off_lat,off_dong,off_hour,off_minute,off_second,no_transfer,total_distance,total_duration,카드번호,노선ID,승차일시,하차일시
0,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,지하철,영등포시장역,126.905118,37.522694,1119074,12,42,8,충무로역,126.993624,37.561135,1102060,13,16,20,0,11000,2052,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,NaN,2.017102e+13,2.017102e+13
1,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,지하철,충무로역,126.993624,37.561135,1102060,16,2,9,중곡역,127.084283,37.565891,1105057,16,36,50,0,9400,2081,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,NaN,2.017102e+13,2.017102e+13
2,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,지하철,중곡역,127.084283,37.565891,1105057,18,22,39,선유도역,126.893294,37.538159,1119062,19,19,46,0,20100,3427,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,NaN,2.017102e+13,2.017102e+13
3,+++19/4InCnN/eZXlK0pT7oFlp3qEbVXHX4NFSiyAN8=,버스,LG전자.문래중,126.885883,37.520390,1119076,16,11,56,문래역,126.894412,37.519063,1119076,16,15,3,0,768,187,+++19/4InCnN/eZXlK0pT7oFlp3qEbVXHX4NFSiyAN8=,11110803.0,2.017102e+13,2.017102e+13
4,+++3lFnkOIxAPXhin+oeJBZJeI8l5KOhgxcjhFqGTlc=,버스,강남구청.강남세무서,127.047293,37.518428,1123078,18,4,57,청담동래미안아파트,127.042912,37.517653,1123078,18,6,30,0,397,93,+++3lFnkOIxAPXhin+oeJBZJeI8l5KOhgxcjhFqGTlc=,11110214.0,2.017102e+13,2.017102e+13


In [112]:
# 취합 데이터
df.to_csv('smartcard_data_merge.csv', index = False, encoding = 'cp949')

# 02. 환승을 판단하고 개별 Trip chain의 Circuity 산출

In [113]:
# !pip install haversine

In [114]:
pd.set_option('display.max_column', 100)

In [115]:
from haversine import haversine

tic = time.time()

TB_card = pd.read_csv("./smartcard_data_merge.csv", encoding = "cp949")
print("Loading", time.time()-tic)

def MyConCat(TB):
    TB_temp1 = TB.shift(1).add_suffix("1")
    TB_temp2 = TB.add_suffix("2")
    TB_temp3 = pd.concat([TB_temp1, TB_temp2], axis = 1)
    return(TB_temp3)

def ftn_ChainID():
    TB_card2 = MyConCat(TB_card)
    bl_card = (TB_card2.card_name1 == TB_card2.card_name2)
    On_Second2 = TB_card2.on_hour2*3600 + TB_card2.on_minute2*60 + TB_card2.on_second2
    Off_Second1 = TB_card2.off_hour1*3600 + TB_card2.off_minute1*60 + TB_card2.off_second1
    bl_wait = ( On_Second2 - Off_Second1 <= 3600 )
    bl_tr = (TB_card2.no_transfer2 - TB_card2.no_transfer1 == 1)
    idx_Transfer = bl_card & bl_wait & bl_tr
    idx_Transfer = 1-idx_Transfer
    ChainID = idx_Transfer.cumsum()
    return(ChainID)
TB_card["ChainID"] = ftn_ChainID()
print("Chain", time.time()-tic)

TB_card["Bus"] = False
TB_card["Subway"] = False
TB_card.loc[TB_card.route_id == "버스", "Bus"] = True
TB_card.loc[TB_card.route_id == "지하철", "Subway"] = True

TB_card = TB_card.groupby("ChainID").agg(
    card_name = ("card_name", "first"),
    Bus = ("Bus", "sum"),
    Subway = ("Subway", "sum"),
    on_area_name = ("on_area_name", "first"),
    on_lon = ("on_lon", "first"),
    on_lat = ("on_lat", "first"),
    on_dong = ("on_dong", "first"),
    on_hour = ("on_hour", "first"),
    on_minute = ("on_minute", "first"),
    on_second = ("on_second", "first"),
    off_area_name = ("off_area_name", "last"),
    off_lon = ("off_lon", "last"),
    off_lat = ("off_lat", "last"),
    off_dong = ("off_dong", "last"),
    off_hour = ("off_hour", "last"),
    off_minute = ("off_minute", "last"),
    off_second = ("off_second", "last"),
    no_transfer = ("no_transfer", "last"),
    total_distance = ("total_distance", "sum"),
    total_time = ("total_duration", "sum"),
    route_id = ("노선ID", "first")
)
print("Agg", time.time() - tic)

TB_card = TB_card.astype({"Bus":"bool", "Subway":"bool"})
TB_card["Haversine"] = TB_card.apply(lambda x : haversine((x["on_lat"], x["on_lon"]), (x["off_lat"], x["off_lon"]), "m"), axis = 1)
TB_card["Circuity"] = TB_card.total_distance / TB_card.Haversine
print("Circuity", time.time() - tic)

TB_card.to_csv("smartcard_Chain.csv", encoding = "cp949", index = False)
print("Export", time.time() - tic)

Loading 43.97846698760986
Chain 65.03752565383911
Agg 76.43853449821472
Circuity 215.7884373664856
Export 297.03302454948425


In [116]:
TB_card.head()

,card_name,Bus,Subway,on_area_name,on_lon,on_lat,on_dong,on_hour,on_minute,on_second,off_area_name,off_lon,off_lat,off_dong,off_hour,off_minute,off_second,no_transfer,total_distance,total_time,route_id,Haversine,Circuity
ChainID,,,,,,,,,,,,,,,,,,,,,,,
1,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,영등포시장역,126.905118,37.522694,1119074,12,42,8,충무로역,126.993624,37.561135,1102060,13,16,20,0,11000,2052,NaN,8897.367673,1.236321
2,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,충무로역,126.993624,37.561135,1102060,16,2,9,중곡역,127.084283,37.565891,1105057,16,36,50,0,9400,2081,NaN,8008.336598,1.173777
3,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,중곡역,127.084283,37.565891,1105057,18,22,39,선유도역,126.893294,37.538159,1119062,19,19,46,0,20100,3427,NaN,17116.783674,1.174286
4,+++19/4InCnN/eZXlK0pT7oFlp3qEbVXHX4NFSiyAN8=,True,False,LG전자.문래중,126.885883,37.520390,1119076,16,11,56,문래역,126.894412,37.519063,1119076,16,15,3,0,768,187,11110803.0,766.539891,1.001905
5,+++3lFnkOIxAPXhin+oeJBZJeI8l5KOhgxcjhFqGTlc=,True,False,강남구청.강남세무서,127.047293,37.518428,1123078,18,4,57,청담동래미안아파트,127.042912,37.517653,1123078,18,6,30,0,397,93,11110214.0,395.878621,1.002833


In [117]:
TB_card.columns

Index(['card_name', 'Bus', 'Subway', 'on_area_name', 'on_lon', 'on_lat',
       'on_dong', 'on_hour', 'on_minute', 'on_second', 'off_area_name',
       'off_lon', 'off_lat', 'off_dong', 'off_hour', 'off_minute',
       'off_second', 'no_transfer', 'total_distance', 'total_time', 'route_id',
       'Haversine', 'Circuity'],
      dtype='object')